In [1]:
from models import EfficientNet3D
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
batch_size = 8
inputs_shape = (batch_size, 45, 6, 6, 6)
labels_shape = (batch_size,)
inputs = torch.randn(inputs_shape).to(device)
labels = torch.randint(low=0, high=26, size=labels_shape).to(device)

cnn_flattened_size = inputs_shape[1] * inputs_shape[2] * inputs_shape[3] * inputs_shape[4]

print("inputs shape", inputs.shape)
print("labels shape", labels.shape)

inputs shape torch.Size([8, 45, 6, 6, 6])
labels shape torch.Size([8])


In [3]:
num_classes = 27
model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': num_classes}, in_channels=1, cnn_flattened_size=cnn_flattened_size, hidden_size=128, task="classification")

In [4]:
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train()

EfficientNet3D(
  (_conv_stem): Conv3dStaticSamePadding(
    1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
  )
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock3D(
      (_depthwise_conv): Conv3dStaticSamePadding(
        32, 32, kernel_size=(3, 3, 3), stride=[2, 2, 2], groups=32, bias=False
        (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
      )
      (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv3dStaticSamePadding(
        32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv3dStaticSamePadding(
        8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv3dStaticSamePadding(
        32, 24, kernel_siz

In [5]:
previous_prediction_1 = torch.randn((batch_size, num_classes))
previous_prediction_2 = torch.randn((batch_size, num_classes))
# Concatenate the previous predictions together along dimension 2
previous_predictions = torch.cat((previous_prediction_1, previous_prediction_2), dim=1).cuda()
print("Previous prediction size outside", previous_predictions.shape)

Previous prediction size outside torch.Size([8, 54])


In [6]:
for epoch in range(1):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs, previous_predictions)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, loss.item()))

print('Finished Training')
print(outputs.shape)
print(outputs.cpu().detach().numpy())
print(outputs.cpu().detach().numpy().sum(axis=1))

[1] loss: 3.289
Finished Training
torch.Size([8, 27])
[[0.07615834 0.01100689 0.00966028 0.02115127 0.01448039 0.07994676
  0.00798631 0.0109573  0.01141784 0.22375706 0.01381616 0.01308022
  0.01128829 0.0269672  0.0101411  0.00728272 0.00967264 0.00992012
  0.01036912 0.2258371  0.0085873  0.0109349  0.13714576 0.01113639
  0.00962162 0.00777645 0.0099005 ]
 [0.00821441 0.02110444 0.07448553 0.0053279  0.00522231 0.00779177
  0.00991091 0.00556232 0.0086254  0.01439389 0.00685301 0.20675309
  0.00821505 0.15900053 0.10241289 0.00529999 0.00703926 0.02396569
  0.10017681 0.00902898 0.02310225 0.00795787 0.00717767 0.01209432
  0.14741933 0.00565931 0.00720508]
 [0.01029351 0.0100377  0.00880965 0.13353598 0.03059577 0.00884243
  0.07929496 0.06673179 0.01041246 0.0090447  0.01387557 0.01192846
  0.20728512 0.00948682 0.00924814 0.12910233 0.00882093 0.00904662
  0.01536203 0.03984724 0.00783116 0.00997204 0.00899438 0.01015579
  0.0087744  0.00895327 0.12371679]
 [0.01806566 0.0467819